In [1]:
import pandas as pd
from pathlib import Path

In [2]:
path = "COVID-19/csse_covid_19_data/csse_covid_19_daily_reports"
all_files = Path(path).glob("*.csv") 

df = pd.concat(map(pd.read_csv, all_files))

In [3]:
df.shape

(2637010, 21)

In [4]:
df.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,...,Combined_Key,Incident_Rate,Case_Fatality_Ratio,Incidence_Rate,Case-Fatality_Ratio,Province/State,Country/Region,Last Update,Latitude,Longitude
0,NaN,NaN,NaN,Afghanistan,2021-05-06 04:20:38,33.93911,67.709953,60797.0,2654.0,53750.0,...,Afghanistan,156.176704,4.365347,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,Albania,2021-05-06 04:20:38,41.15330,20.168300,131419.0,2403.0,113468.0,...,Albania,4566.648134,1.828503,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,Algeria,2021-05-06 04:20:38,28.03390,1.659600,123272.0,3299.0,85869.0,...,Algeria,281.115320,2.676196,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,Andorra,2021-05-06 04:20:38,42.50630,1.521800,13340.0,127.0,12884.0,...,Andorra,17265.255937,0.952024,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,Angola,2021-05-06 04:20:38,-11.20270,17.873900,27529.0,618.0,24492.0,...,Angola,83.760651,2.244905,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
def remove_small_countries(df):
    countries = df.groupby("Country_Region").last().query('Confirmed > 100000').index.to_list()
    return df[df.Country_Region.isin(countries)]

In [6]:
def report_shape(df):
    print(df.shape)
    return df

In [7]:
(df
 .assign(Last_Update = pd.to_datetime(df["Last_Update"]).dt.date)
 .loc[:, ["Country_Region", "Last_Update", "Confirmed"]]
 .dropna()
 .groupby(["Country_Region", "Last_Update"])
 .sum()
 .reset_index()
 .set_index("Last_Update")
 .sort_index()
 .pipe(remove_small_countries)
 .pipe(report_shape)
 .to_parquet("tmp/data.parquet")
)

(80185, 2)
